# BIDS mover

The goal of this notebook is to move raw, unpreprocessed files (for instance MPRAGE of Freesurfer nifti files) into the [BIDS](https://bids.neuroimaging.io/) format.

Steps:
1. Create a table with the new and old location of all files;
2. Move the files to the new location.

In [1]:
import pandas as pd
import os 
from os.path import join 
import glob
import multiprocessing as mp

DATA_DIR = "/ritter/share/data/IMAGEN"

## 1. Create csv with old and new path 

In [2]:
# List (for instance) all Freesurfer brainmask files in the raw data
files = glob.glob(join(DATA_DIR, "IMAGEN_RAW/2.7/*/imaging/freesurfer/*/mri/brainmask.mgz"), recursive=True)

In [3]:
# New df for storing everything about the subjects
df = pd.DataFrame(columns = ["ID", "time_point", "anat_func", "file_name", "path", "old_path", "dir"])

for i in range(len(files)): 
    file = files[i]
    x = file.split("/")
    
    subject = x[10]
    time_point = x[7]
    
    new_file_name = "sub-{}_ses-{}_fs-brainmask_T1w.mgz".format(subject, time_point)
    new_dir = join(DATA_DIR, "IMAGEN_freesurfer_BIDS", "sub-{}".format(subject), time_point, "anat")
    new_path = join(new_dir, new_file_name)
    
    
    df = df.append({
        "path" : new_path,
        "old_path" : file, 
        "ID" : subject, 
        "file_name" : x[-1], 
        "time_point" : time_point, 
        "anat_func" : "anat", 
        "dir" : new_dir
    }
        , ignore_index=True)

In [4]:
df.time_point.value_counts()

BL     2088
FU2    1413
FU3    1179
Name: time_point, dtype: int64

In [5]:
# Save the obtained table as a participants.csv file in the new BIDS folder
df.to_csv(join(DATA_DIR, "IMAGEN_freesurfer_BIDS", "participants_brainmask.csv"))

## 2. Move the files into their new BIDS directories

In [23]:
old_paths = df.old_path.tolist()
paths = df.path.tolist()

def wrapper(i):
    new_dir = os.path.dirname(paths[i])
    if not os.path.exists(new_dir):
        os.makedirs(new_dir)
    os.system("cp {} {}".format(old_paths[i], paths[i]))

with mp.Pool(5) as p:
    p.map(wrapper, range(len(paths)))

In [24]:
# Check if everything went right 

for i, path in enumerate(paths):
    if not os.path.isfile(path):
        print("Could not find {}".format(path))
        
        
print("Checked {} files.".format(i+1))

Checked 4680 files.
